# [Best Position for a Service Centre](https://leetcode.com/problems/best-position-for-a-service-centre/)
- **Given**: Array `A`
    + which $A[i] = \{x_i, y_i\}$, represent the position of a customer on a 2D map
- Find a place `{x_center, y_center}` (float) to build a service centre, such that, **the sum of the euclidean distances to all customers is minimum**

$$\text{Min } f(x_\text{center}, y_\text{center}) = \sum\limits_{i=0}^{N-1}\sqrt{\left( x_\text{center} - x_i \right)^2 + \left( y_\text{center} - y_i \right)^2}$$

- **Return**: the min total distance

#### Constraints
- $1 \leq N \leq 50$
- $0 \leq x_i, y_i \leq 100$

#### Example 1

```
Input: positions = [[0,1],[1,0],[1,2],[2,1]]
Output: 4.00000
Explantion: Build the centre at {1.0, 1.0} 
```

<img src="./img/12.jpg" width="300"/>

#### Example 2

```
Input: positions = [[1,1],[3,3]]
Output: 2.82843
Explantion: Build the centre at {2.0, 2.0} 
```

<img src="./img/13.jpg" width="300"/>

## Solution 1 - Scan search
- Divide the grid into sub grids, find the min cell
- Increase the resolution, Zoom-in this cell -> search for the min subcell

<img src="./img/14.png" width="300"/>

```Cpp
class Solution {
public:
    vector<vector<int>> A;
    double f(double x_center, double y_center) {
        double total_dist = 0.0;
        for(auto &pos: A) {
            int x = pos[0];
            int y = pos[1];

            total_dist += sqrt((x-x_center)*(x-x_center) + (y-y_center)*(y-y_center)); 
        }
        return total_dist;
    }
    double getMinDistSum(const vector<vector<int>> &positions) {
        A = positions;

        double x_min = 0.0, x_max = 100.0;
        double y_min = 0.0, y_max = 100.0;
        double x_center = 0.0, y_center = 0.0;
        double ans = DBL_MAX;
        vector<double> deltas({1.0, 0.1, 0.01, 0.001, 0.0001, (double)1e-5, (double)1e-6});

        for(double delta: deltas) {
            // Search for the min cell in current resolution
            for(double x=x_min; x<=x_max; x+=delta) {
                for(double y=y_min; y<=y_max; y+=delta) {

                    // Relax
                    double dist = f(x, y);
                    if(dist < ans) {
                        ans = dist;
                        x_center = x;
                        y_center = y;
                    }
                }
            }

            // Update next Zoom-in cell
            x_min = max(0.0, x_center - delta);
            x_max = min(100.0, x_center + delta);

            y_min = max(0.0, y_center - delta);
            y_max = min(100.0, y_center + delta);
        }
        return ans;
    }
};
```

## Solution 2 - gradient descent
- Start with a point (centroid) moving toward the optimum point with decay learning rate

```Cpp
class Solution {
public:
    vector<vector<int>> A;
    double f(double x_center, double y_center) {
        double total_dist = 0.0;
        for(auto &pos: A) {
            int x = pos[0];
            int y = pos[1];

            total_dist += sqrt((x-x_center)*(x-x_center) + (y-y_center)*(y-y_center)); 
        }
        return total_dist;
    }
    double getMinDistSum(const vector<vector<int>> &positions) {
        A = positions;

        // Start with centroid
        double x_center = 0.0, y_center = 0.0;
        for(auto &pos: A) {
            int x = pos[0];
            int y = pos[1];

            x_center += x;
            y_center += y;
        }
        x_center /= A.size();
        y_center /= A.size();

        // Gradient descent
        double ans = DBL_MAX;
        vector<double> learning_rates({1.0, 0.1, 0.01, 0.001, 0.0001, (double)1e-5, (double)1e-6});
        for(double lr: learning_rates) {
            bool decay_lr = false;
            while(decay_lr == false) {
                // Search 4 neighboring dirs
                for(auto [x,y]: vector<pair<double,double>>{
                        {x_center + lr, y_center + lr},
                        {x_center - lr, y_center + lr},
                        {x_center + lr, y_center - lr},
                        {x_center - lr, y_center - lr} }) {

                    // Relax
                    double dist = f(x, y);
                    if(dist < ans) {
                        ans = dist;
                        x_center = x;
                        y_center = y;
                        decay_lr = false;
                        break;
                    } else decay_lr = true;
                }
            }
        }
        return ans;
    }
};
```